In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from itertools import product
from datetime import date
from path import Path
import pandas as pd
import random

import os
os.environ['NUMEXPR_MAX_THREADS'] = '32'

from mlbt.run_bt import run_bt
from mlbt.feature_eng import define_feature_configs
from mlbt.load_data import LENGTH_RANKING


DATA_DIR = Path("~/Dropbox/algotrading/data").expanduser()
s_csv = pd.read_csv(DATA_DIR / "symbols.csv", index_col="iqsymbol")

# Code to execute the pipeline inside a multiprocessing.Pool
# This creates the payloads that "run_frontend_payloads.ipynb" can use to generate the
# frontend payloads (.json loaded by the frontend)
ranking = ["equity_index", ""]


symbol_groups = list(
    {
        "agriculture": "Agriculture",
        "currency": "Currency",
        "energy": "Energy",
        "equity_index": "Equity Index",
        "interest_rate": "Interest Rate",
        "metals": "Metals",
    }.keys()
)

features = define_feature_configs()
print(len(features))
c_s = (len(features) // 32) + 1 
c_s = 1 
print('c_s', c_s)
data_sim = {
    "start_date": [date(2000, 1, 1)],
    "end_date": [date(2021, 1, 1)],
#     "symbol_groups": [symbol_groups],
#     "symbol_groups": [[x] for x in symbol_groups],
#     "symbol_groups": [['equity_index'], ['interest_rate']],
#     "symbols": [['@NQ#C']],
    "symbols": [[x] for x in LENGTH_RANKING],
    "bar_type": ["dollar"],
    "binarize": ["fixed_horizon"],
    "binarize_params": [10, 25, 50, 100],
#     "binarize_params": [10, 25, 100],
#     "binarize_params": [50],
    "alpha": ["none"],
#     "features": [features[i:i+c_s] for i in range(0, len(features), c_s)],
    "features": [features],
#     "classifier": ["random_forest", "xgboost", "lgbm", "knn", "dummy"],
#     "classifier": ["random_forest", "xgboost", "lgbm", "dummy"],
#     "classifier": ["random_forest", "xgboost", "dummy"],
#     "classifier": ["tpot"],
    "classifier": ["lgbm"],
    "num_threads": [32], # how many parallel hyperopti 
    "n_jobs": [32], # how many parallel infer
    "num_threads": [1], # how many parallel hyperopti 
    "n_jobs": [1], # how many parallel infer
    "feat_imp_n_estimators": [1000],
    "feat_imp_cv": [10],
    "func": [run_bt],
#     "feature_calc_only": [True],
    "feature_imp_only": [True],
    "check_completed": [False],
#     "optimize_hypers": [False],
#     "reuse_hypers": [False],
#     "load_from_disk": [False],
#     "skip_feature_imp": [True],
}

c:\users\doda\anaconda3\envs\fincl\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
c:\users\doda\anaconda3\envs\fincl\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
2020-02-15 02:24:33,248 Loaded backend module://ipykernel.pylab.backend_inline version unknown.


91
c_s 1


In [3]:
datas = list(dict(zip(data_sim.keys(), x)) for x in product(*data_sim.values()))
datas
for data in datas:
    data['features'] = list(sorted(data['features'], key=lambda x:random.random()))


In [4]:
len(datas)

192

In [5]:
[x['features'][0] for x in datas]

[{'name': 'rolling_stdev', 'window': 250, 'symbol': {'name': 'volratio'}},
 {'name': 'amihud', 'window': 25},
 {'name': 'rolling_stdev',
  'window': 50,
  'symbol': {'name': 'rolling_stdev',
   'window': 50,
   'symbol': {'name': 'log_ret'}}},
 {'name': 'rolling_stdev', 'window': 25, 'symbol': {'name': 'log_ret'}},
 {'name': 'rolling_stdev',
  'window': 50,
  'symbol': {'name': 'rollimp', 'window': 50}},
 {'name': 'rolling_stdev',
  'window': 25,
  'symbol': {'name': 'rollimp', 'window': 25}},
 {'name': 'ewm_mean', 'com': 250, 'symbol': {'name': 'volratio'}},
 {'name': 'ewm_stdev', 'com': 25, 'symbol': {'name': 'ffd', 'd': 0.3}},
 {'name': 'roll', 'window': 250},
 {'name': 'ewm_stdev', 'com': 250, 'symbol': {'name': 'ffd', 'd': 0.3}},
 {'name': 'entropy',
  'method': 'konto',
  'encoding': 'sigma',
  'window': 250,
  'symbol': {'name': 'log_ret'},
  'konto_len': 26},
 {'name': 'rolling_stdev',
  'window': 50,
  'symbol': {'name': 'rolling_stdev',
   'window': 50,
   'symbol': {'name': 

In [6]:
from mlbt.multiprocess import process_jobs

In [7]:
num_threads = max(1, min(32, len(datas)) - 0)
num_threads
      

32

In [ ]:
process_jobs(datas, num_threads=num_threads)

In [ ]:
# for data in datas:
#     run_bt(**data)

In [ ]:
%debug

In [ ]:
1

In [ ]:
print '*' * 100000000